# Maps Kevin's data onto reactome pathways

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import glob

import pylab as plt
import matplotlib
from IPython.display import display, HTML

import numpy as np
from scipy.sparse import lil_matrix
import pandas as pd
from pandas import HDFStore

import seaborn as sns
from IPython.display import display
    
from collections import defaultdict

from ipywidgets import FloatProgress

%matplotlib inline

In [4]:
import sys
sys.path.append('../linker')

from reactome import ensembl_to_uniprot, uniprot_to_reaction, compound_to_reaction, \
    reaction_to_metabolite_pathway, get_reaction_ids, get_reaction_df, get_reaction_entities, \
    produce_kegg_dict, get_all_pathways_formulae


## Load data

In [5]:
basedir = '/Users/joewandy/Dropbox/Analysis/omics_integration'

In [6]:
time = 7
parasite = 'INFEC'
treatment = 'Unsorted'

filename = os.path.join(basedir, 'data/data_%d_%s_%s.h5' % (time, parasite, treatment))
hdf = HDFStore(filename, complevel=9, complib='bzip2')

pvt = hdf['peak_vs_transcript'].replace([np.inf, -np.inf], np.nan).fillna(0)
# pvp = hdf['peak_vs_peak']
# tvt = hdf['transcript_vs_transcript']

hdf.close()

In [7]:
pvt.columns

Index(['ENSMUSG00000000001', 'ENSMUSG00000000028', 'ENSMUSG00000000031',
       'ENSMUSG00000000049', 'ENSMUSG00000000056', 'ENSMUSG00000000058',
       'ENSMUSG00000000078', 'ENSMUSG00000000085', 'ENSMUSG00000000088',
       'ENSMUSG00000000093',
       ...
       'ENSMUSG00000110391', 'ENSMUSG00000110393', 'ENSMUSG00000110397',
       'ENSMUSG00000110399', 'ENSMUSG00000110404', 'ENSMUSG00000110405',
       'ENSMUSG00000110410', 'ENSMUSG00000110414', 'ENSMUSG00000110419',
       'ENSMUSG00000110424'],
      dtype='object', length=21794)

In [8]:
species = 'Mus musculus'

## Maps ENSEMBL IDs to UniProt IDs

In [9]:
ens_id = pvt.columns[0]
print(ens_id)

ENSMUSG00000000001


In [10]:
print(len(pvt.columns))
ens_ids = pvt.columns.values.tolist()

21794


In [11]:
transcript_mapping = ensembl_to_uniprot(ens_ids, species, show_progress_bar=True)

FloatProgress(value=0.0, max=21794.0)

In [12]:
ensembl_ids = list(transcript_mapping.keys())[0:10]
for ensembl_id in ensembl_ids:
    print(ensembl_id, transcript_mapping[ensembl_id])

In [13]:
total_mapped = len(transcript_mapping)
total_genes = len(ens_ids)
certain = []
for ensembl_id in transcript_mapping:
    if len(transcript_mapping[ensembl_id]) == 1:
        certain.append(transcript_mapping[ensembl_id][0])

uncertain = total_mapped - len(certain)
print(len(certain), uncertain, total_mapped, total_genes)

In [33]:
from bioservices import Ensembl

In [79]:
ensembl_ids = list(transcript_mapping.keys())
ens = Ensembl()

In [85]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

ensembl_lookup = {}
for x in batch(ensembl_ids, 1000):
    batch_ids = [i for i in x]
    print(len(batch_ids))
    lookup = ens.post_lookup_by_id(identifiers=batch_ids)
    ensembl_lookup.update(lookup)

1000
1000
1000
1000
1000
1000
600


In [87]:
ensembl_lookup['ENSMUSG00000002100']

{'assembly_name': 'GRCm38',
 'biotype': 'protein_coding',
 'db_type': 'core',
 'description': 'myosin binding protein C, cardiac [Source:MGI Symbol;Acc:MGI:102844]',
 'display_name': 'Mybpc3',
 'end': 91136516,
 'id': 'ENSMUSG00000002100',
 'logic_name': 'ensembl_havana_gene',
 'object_type': 'Gene',
 'seq_region_name': '2',
 'source': 'ensembl_havana',
 'species': 'mus_musculus',
 'start': 91118144,
 'strand': 1,
 'version': 15}

## Map UniProt IDs to Reactions

In [88]:
protein_mapping = uniprot_to_reaction(certain, species, show_progress_bar=True)

FloatProgress(value=0.0, max=5987.0)

In [89]:
for uniprot_id in certain[0:3]:
    print(uniprot_id, protein_mapping[uniprot_id])
    print

P14753 [{'reaction_id': 'R-MMU-209310', 'reaction_name': 'Murine JAK2 binds to the Erythropoietin receptor, EpoR'}]
Q60754 [{'reaction_id': 'R-MMU-2247510', 'reaction_name': 'MARCO:ligand is endocytosed'}, {'reaction_id': 'R-MMU-2173783', 'reaction_name': 'Marco binds ligands'}]
P30204 [{'reaction_id': 'R-MMU-2507854', 'reaction_name': 'MSR1:ligand (SCARA1:ligand, SR-A:ligand) is endocytosed'}, {'reaction_id': 'R-MMU-2173779', 'reaction_name': 'Msr1 (Scara1) Binds Ligands'}]


In [157]:
from bioservices import UniProt

In [158]:
uniprot = UniProt()
uniprot_ids = certain
res = uniprot.retrieve(uniprot_ids)
print(len(res))

5987


In [159]:
protein_metadata = {}
for r in res:
    for key in r['accession']:
        protein_id = key.contents[0]    
        for x in r['recommendedname']:
            tag = x.find('shortname')
            if tag is None:
                tag = x.find('fullname')
            label = tag.contents[0]
            protein_metadata[protein_id] = {'display_name': label}

In [160]:
for protein_id in protein_metadata:
    print(protein_id, protein_metadata[protein_id])

P14753 {'display_name': 'EPO-R'}
Q63852 {'display_name': 'EPO-R'}
Q60754 {'display_name': 'Macrophage receptor MARCO'}
P30204 {'display_name': 'Macrophage scavenger receptor types I and II'}
E9QNQ5 {'display_name': 'Macrophage scavenger receptor types I and II'}
Q923G0 {'display_name': 'Macrophage scavenger receptor types I and II'}
Q9QZ56 {'display_name': 'Macrophage scavenger receptor types I and II'}
Q8BV57 {'display_name': 'Soluble scavenger receptor cysteine-rich domain-containing protein SSC5D'}
B2RUB0 {'display_name': 'Soluble scavenger receptor cysteine-rich domain-containing protein SSC5D'}
C6ZFQ2 {'display_name': 'Soluble scavenger receptor cysteine-rich domain-containing protein SSC5D'}
Q3TMW2 {'display_name': 'Soluble scavenger receptor cysteine-rich domain-containing protein SSC5D'}
Q9JKR6 {'display_name': 'GRP-170'}
Q3TAL1 {'display_name': 'GRP-170'}
Q3TZD0 {'display_name': 'GRP-170'}
Q3U1U2 {'display_name': 'GRP-170'}
Q64139 {'display_name': 'GRP-170'}
Q80X75 {'display_n

Q9CR95 {'display_name': 'Adaptin ear-binding coat-associated protein 1'}
Q3TH13 {'display_name': 'Adaptin ear-binding coat-associated protein 1'}
Q78JW3 {'display_name': 'Adaptin ear-binding coat-associated protein 1'}
Q8C4P1 {'display_name': 'Adaptin ear-binding coat-associated protein 1'}
Q6PHU5 {'display_name': 'Sortilin'}
A2AEE8 {'display_name': 'Sortilin'}
Q3UHE2 {'display_name': 'Sortilin'}
Q8K043 {'display_name': 'Sortilin'}
Q9QXW6 {'display_name': 'Sortilin'}
A3KGB4 {'display_name': 'TBC1 domain family member 8B'}
B9EJW4 {'display_name': 'TBC1 domain family member 8B'}
O54818 {'display_name': 'mD53'}
Q62393 {'display_name': 'mD52'}
Q545M5 {'display_name': 'mD52'}
Q8CCU1 {'display_name': 'mD52'}
Q8K564 {'display_name': 'mD52'}
Q99KP8 {'display_name': 'mD52'}
Q80TZ3 {'display_name': 'Putative tyrosine-protein phosphatase auxilin'}
B1B0B9 {'display_name': 'Putative tyrosine-protein phosphatase auxilin'}
Q6P2K9 {'display_name': 'Putative tyrosine-protein phosphatase auxilin'}
Q8C7L

O70480 {'display_name': 'VAMP-4'}
Q9D095 {'display_name': 'VAMP-4'}
Q5SPW0 {'display_name': 'Vacuolar protein sorting-associated protein 54'}
Q8BPB3 {'display_name': 'Vacuolar protein sorting-associated protein 54'}
Q8CFZ7 {'display_name': 'Vacuolar protein sorting-associated protein 54'}
Q8CHL5 {'display_name': 'Vacuolar protein sorting-associated protein 54'}
Q8R3R4 {'display_name': 'Vacuolar protein sorting-associated protein 54'}
Q8R3X1 {'display_name': 'Vacuolar protein sorting-associated protein 54'}
Q3UVL4 {'display_name': 'Vacuolar protein sorting-associated protein 51 homolog'}
Q0VF89 {'display_name': 'Vacuolar protein sorting-associated protein 51 homolog'}
Q0VF90 {'display_name': 'Vacuolar protein sorting-associated protein 51 homolog'}
Q3UK22 {'display_name': 'Vacuolar protein sorting-associated protein 51 homolog'}
Q9CXM6 {'display_name': 'Vacuolar protein sorting-associated protein 51 homolog'}
Q8C754 {'display_name': 'Vacuolar protein sorting-associated protein 52 homolo

B1AWI4 {'display_name': 'RAB6A-GEF complex partner protein 2'}
Q6ZQF1 {'display_name': 'RAB6A-GEF complex partner protein 2'}
Q69ZJ7 {'display_name': 'RAB6A-GEF complex partner protein 1'}
A4GZ02 {'display_name': 'RAB6A-GEF complex partner protein 1'}
Q3U0Q6 {'display_name': 'RAB6A-GEF complex partner protein 1'}
Q3UWK1 {'display_name': 'RAB6A-GEF complex partner protein 1'}
Q3U1T9 {'display_name': 'DENN domain-containing protein 1B'}
B2RUR7 {'display_name': 'DENN domain-containing protein 1B'}
E9Q246 {'display_name': 'DENN domain-containing protein 1B'}
Q8CG96 {'display_name': 'DENN domain-containing protein 1B'}
Q9D5B9 {'display_name': 'DENN domain-containing protein 1B'}
Q8K382 {'display_name': 'DENN domain-containing protein 1A'}
A0MCI0 {'display_name': 'DENN domain-containing protein 1A'}
A2ALU2 {'display_name': 'DENN domain-containing protein 1A'}
Q8K2L8 {'display_name': 'Trafficking protein particle complex subunit 12'}
E9QLQ6 {'display_name': 'Trafficking protein particle compl

Q3UCG4 {'display_name': 'Mucolipin-2'}
Q8K2T6 {'display_name': 'Mucolipin-2'}
Q9CQD3 {'display_name': 'Mucolipin-2'}
Q9JLV2 {'display_name': 'Trp4-associated protein'}
Q920J6 {'display_name': 'Trp4-associated protein'}
Q99L03 {'display_name': 'Trp4-associated protein'}
O35454 {'display_name': 'Chloride transport protein 6'}
Q9WVD4 {'display_name': 'H(+)/Cl(-) exchange transporter 5'}
B1AXN0 {'display_name': 'H(+)/Cl(-) exchange transporter 5'}
Q61418 {'display_name': 'H(+)/Cl(-) exchange transporter 4'}
Q2TAX6 {'display_name': 'H(+)/Cl(-) exchange transporter 4'}
Q64347 {'display_name': 'ClC-1'}
Q8BZ41 {'display_name': 'ClC-1'}
Q9QVY5 {'display_name': 'ClC-1'}
Q9WUB6 {'display_name': 'Chloride channel Kb'}
A2ADB6 {'display_name': 'Chloride channel Kb'}
Q8VCF8 {'display_name': 'Chloride channel Kb'}
Q9R0A1 {'display_name': 'ClC-2'}
Q0VBC2 {'display_name': 'ClC-2'}
Q9WUJ9 {'display_name': 'ClC-2'}
Q8BH79 {'display_name': 'Anoctamin-10'}
Q4FZD2 {'display_name': 'Anoctamin-10'}
Q8BMS8 {'di

...

Q8R4K2 {'display_name': 'IRAK-4'}
Q80WW1 {'display_name': 'IRAK-4'}
Q62406 {'display_name': 'IRAK'}
B1AUW4 {'display_name': 'IRAK'}
Q6Y3Z5 {'display_name': 'IRAK'}
Q6Y3Z6 {'display_name': 'IRAK'}
P14719 {'display_name': 'Interleukin-1 receptor-like 1'}
Q05208 {'display_name': 'Interleukin-1 receptor-like 1'}
Q61730 {'display_name': 'IL-1 receptor accessory protein'}
Q3UVZ1 {'display_name': 'IL-1 receptor accessory protein'}
Q8VCB9 {'display_name': 'IL-1 receptor accessory protein'}
P22366 {'display_name': 'Myeloid differentiation primary response protein MyD88'}
O35916 {'display_name': 'Myeloid differentiation primary response protein MyD88'}
P41245 {'display_name': 'MMP-9'}
Q06788 {'display_name': 'MMP-9'}
Q80XI8 {'display_name': 'MMP-9'}
Q9DC02 {'display_name': 'MMP-9'}
A8E0Y8 {'display_name': 'IgSF2'}
A8E0Y9 {'display_name': 'IgSF2'}
Q3U1F9 {'display_name': 'Phosphoprotein associated with glycosphingolipid-enriched microdomains 1'}
Q8BFS1 {'display_name': 'Phosphoprotein associated 

Q9R1D8 {'display_name': 'mPDK1'}
Q9R215 {'display_name': 'mPDK1'}
Q9WVC6 {'display_name': 'Serine/threonine-protein kinase Sgk1'}
Q3TJN4 {'display_name': 'Serine/threonine-protein kinase Sgk1'}
Q3UKD0 {'display_name': 'Serine/threonine-protein kinase Sgk1'}
Q3UKF2 {'display_name': 'Serine/threonine-protein kinase Sgk1'}
Q3V1V1 {'display_name': 'Serine/threonine-protein kinase Sgk1'}
Q6NS85 {'display_name': 'Serine/threonine-protein kinase Sgk1'}
P23804 {'display_name': 'E3 ubiquitin-protein ligase Mdm2'}
Q61040 {'display_name': 'E3 ubiquitin-protein ligase Mdm2'}
Q64330 {'display_name': 'E3 ubiquitin-protein ligase Mdm2'}
Q91XK7 {'display_name': 'E3 ubiquitin-protein ligase Mdm2'}
P04184 {'display_name': 'Thymidine kinase, cytosolic'}
Q58EU0 {'display_name': 'Thymidine kinase, cytosolic'}
Q8C3P8 {'display_name': 'Thymidine kinase, cytosolic'}
Q921J8 {'display_name': 'Thymidine kinase, cytosolic'}
Q80U70 {'display_name': 'Polycomb protein Suz12'}
B1AQE5 {'display_name': 'Polycomb protei

Q3TT30 {'display_name': 'RNA polymerases I, II, and III subunit ABC3'}
P62876 {'display_name': 'RNA polymerases I, II, and III subunit ABC5'}
P52436 {'display_name': 'RNA polymerases I, II, and III subunit ABC5'}
Q3UF14 {'display_name': 'RNA polymerases I, II, and III subunit ABC5'}
P97760 {'display_name': 'RNA polymerase II subunit 3'}
Q99M46 {'display_name': 'RNA polymerase II subunit 3'}
Q63871 {'display_name': 'RNA polymerases I, II, and III subunit ABC4'}
Q80UW8 {'display_name': 'RNA polymerases I, II, and III subunit ABC1'}
Q8CFI7 {'display_name': 'DNA-directed RNA polymerase II subunit RPB2'}
Q80Z64 {'display_name': 'Homeobox protein NANOG'}
Q6SMR1 {'display_name': 'Homeobox protein NANOG'}
Q7TN85 {'display_name': 'Homeobox protein NANOG'}
Q64249 {'display_name': 'Nuclear receptor subfamily 6 group A member 1'}
A2AQL8 {'display_name': 'Nuclear receptor subfamily 6 group A member 1'}
A2AQM1 {'display_name': 'Nuclear receptor subfamily 6 group A member 1'}
Q687Z3 {'display_name': 

Q9R1A8 {'display_name': 'E3 ubiquitin-protein ligase RFWD2'}
Q8BRK8 {'display_name': 'AMPK subunit alpha-2'}
B1ASQ8 {'display_name': 'AMPK subunit alpha-2'}
Q3UYM4 {'display_name': 'AMPK subunit alpha-2'}
Q5EG47 {'display_name': 'AMPK subunit alpha-1'}
O54950 {'display_name': 'AMPK gamma1'}
Q5PRE8 {'display_name': 'AMPK gamma1'}
Q91WG5 {'display_name': 'AMPK gamma2'}
E9QK80 {'display_name': 'AMPK gamma2'}
Q6V7V5 {'display_name': 'AMPK gamma2'}
Q9R078 {'display_name': 'AMPK subunit beta-1'}
Q91YN6 {'display_name': 'AMPK subunit beta-1'}
Q6PAM0 {'display_name': 'AMPK subunit beta-2'}
Q91Z50 {'display_name': 'FEN-1'}
Q9CQ36 {'display_name': 'DNA polymerase epsilon subunit 4'}
Q3TJ13 {'display_name': 'DNA polymerase epsilon subunit 4'}
Q9WVF7 {'display_name': 'DNA polymerase epsilon catalytic subunit A'}
E9QKW1 {'display_name': 'DNA polymerase epsilon catalytic subunit A'}
Q9QX50 {'display_name': 'DNA polymerase epsilon catalytic subunit A'}
O54956 {'display_name': 'DNA polymerase epsilon 

Q3TST2 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q3UFQ6 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q3URH7 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q5DU32 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q7TPR5 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q80VV7 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q9QYT1 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q9Z168 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q9Z171 {'display_name': 'E3 ubiquitin-protein ligase HERC2'}
Q80XJ2 {'display_name': 'E3 ubiquitin-protein ligase RNF168'}
Q9JM05 {'display_name': 'E3 SUMO-protein ligase PIAS4'}
Q8R165 {'display_name': 'E3 SUMO-protein ligase PIAS4'}
P48754 {'display_name': 'Breast cancer type 1 susceptibility protein homolog'}
A2A4Q4 {'display_name': 'Breast cancer type 1 susceptibility protein homolog'}
Q60957 {'display_name': 'Breast cancer type 1 susceptibility protein homolog'}

P97397 {'display_name': 'Mitotic checkp

## Map KEGG Compound IDs to Reactions

Load a dataframe containing the KEGG IDs of identified compounds only

In [ ]:
peak_df = pd.read_csv(os.path.join(basedir, 'data/my_analysis_peaks.csv'), index_col=0)

In [ ]:
peak_df.shape

In [ ]:
peak_df.head()

In [ ]:
compound_ids = sorted(list(set(peak_df['identifier'].values.tolist())))

In [ ]:
compound_mapping = compound_to_reaction(compound_ids, species, show_progress_bar=True)

In [ ]:
print(list(compound_mapping.keys())[0:10])

## Map Reactions to Pathways

In [ ]:
protein_reactions = list(set(get_reaction_ids(protein_mapping)))
compound_reactions = list(set(get_reaction_ids(compound_mapping)))
combined_reactions = list(set(protein_reactions + compound_reactions))

print(len(protein_reactions))
print(len(compound_reactions))
print(len(combined_reactions))

In [ ]:
print(combined_reactions[0:10])

This is a map of reactions to pathways that are under the top-level biochemical ('Metabolism') pathways

In [ ]:
pathway_mapping = reaction_to_metabolite_pathway(combined_reactions, species, show_progress_bar=True, leaf=True)
print(len(pathway_mapping))
print(pathway_mapping['R-MMU-2395768'])

In [ ]:
kegg_location = os.path.join(basedir, 'data/kegg/kegg.xml')
kegg_dict = produce_kegg_dict(kegg_location, 'id')

In [ ]:
reaction_df = get_reaction_df(transcript_mapping, protein_mapping, compound_mapping, pathway_mapping, species)
reaction_df.sort_values(by=['compound_coverage', 'protein_coverage'], inplace=True, ascending=False)
print(reaction_df.shape)

In [ ]:
reaction_df

In [ ]:
reaction_df.to_csv('reaction_df.tsv', sep='\t', index=False)

### How many unique pathways are linked to reactions?

In [ ]:
reaction_df['pathway_names'].values

In [ ]:
reaction_df['pathway_ids'].values

In [ ]:
pathway_ids_to_names = {}
for ids, names in zip(reaction_df['pathway_ids'].values, reaction_df['pathway_names'].values):
    pathway_ids = ids.split(':')
    pathway_names = names.split(':')
    for pid, pn in zip(pathway_ids, pathway_names):
        pathway_ids_to_names[pid] = pn

In [ ]:
data = []
for pid, name in pathway_ids_to_names.items():
    row = (pid, name.lower(), )
    data.append(row)
    
pathway_df = pd.DataFrame(data, columns=['pathway_id', 'pathway_name'])
pathway_df = pathway_df.sort_values(by='pathway_name').reset_index(drop=True)
display(pathway_df)

### Hypergeometric test

In [ ]:
pw_f, pathway_id_to_name = get_all_pathways_formulae(species)

In [ ]:
detected = set(peak_df[['formula']].values.flatten())
print(detected, len(detected))

In [ ]:
data = []
for pathway_id in pw_f:
    
    pathway_name = pathway_id_to_name[pathway_id]
    
    formulae = pw_f[pathway_id]
    formulae_count = len(formulae)
    formulae_str = ','.join(sorted(formulae))
    
    detected_f = set([x for x in formulae if x in detected])
    detected_count = len(detected_f)
    detected_str = ','.join(sorted(detected_f))
    
    row = [pathway_id, pathway_name, formulae_str, formulae_count, detected_str, detected_count]
    data.append(row)
    
all_pathway_df = pd.DataFrame(data, columns=[
    'pathway_id', 'pathway_name', 
    'formula', 'formula_count',
    'detected', 'detected_count'])
all_pathway_df.set_index('pathway_id', drop=True, inplace=True)

Compute hypergeometric p-values

In [ ]:
from scipy.stats import hypergeom

In [ ]:
# M = the number of unique formula in all pathways in Reactome
M = len(set(','.join(all_pathway_df['formula'].values).split(',')))

# N = the number of unique formula in all pathways in the dataset
N = len(set(','.join(all_pathway_df['detected'].values).split(',')))

SMOOTHING = 1

data = []
for idx, row in all_pathway_df.iterrows():
        
    # k = the number of unique formula in the pathway of interest in the dataset
    k = row['detected_count']
    
    # n = the number of unique formula in the pathway of interest
    n = row['formula_count'] + SMOOTHING
        
    p_value = hypergeom.sf(k, M, n, N)
    assert p_value > 0
    new_row = [idx, p_value]
    data.append(new_row)

p_value_df = pd.DataFrame(data, columns=[
    'pathway_id', 'p_value'])
p_value_df.set_index('pathway_id', drop=True, inplace=True)

In [ ]:
combined = pd.concat([all_pathway_df, p_value_df], axis=1)
combined = combined.sort_values(by='p_value', ascending=True).reset_index(drop=True)
display(combined)
combined.to_csv('pathway_df.csv', index=False, encoding='utf-8')